In [3]:
import pandas as pd
import numpy as np

In [68]:
data = pd.read_csv('../Data/allSongs.csv', index_col=0)
data.Title = data['Title'].str.lower()
data.Artist = data['Artist'].str.lower()
data.head()

,Artist,Title,Top100
0,linkin park,in the end,0
1,red hot chili peppers,by the way,0
2,evanescence,bring me to life,0
3,thirty seconds to mars,the kill (bury me),0
4,muse,uprising,0


### Spotify URI for songs

In [4]:
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm_notebook
import spotipy
import spotipy.util as util
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

In [5]:

client_id = '9d686c7d0833427db57420593ab7038b'

client_secret = '2e4ab33713e74b8990c8f1cf2da115f0'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [71]:
titles = list(data.Title)
artists = list(data.Artist)
spotify_uri = list()
errors = list()
print(len(titles))

17499


In [72]:
def get_spotify_uri(title, artist):
    title_clean = re.sub(r"[,.;@#?!&$%()]+", ' ', title)
    title_clean = re.sub('\s+', ' ', title_clean).strip()
    artist_clean = re.sub('\s+', ' ', artist).strip()
    
    query = title_clean + " " + artist_clean
    
    search = sp.search(q=query, limit=50, offset=0, type='track', market='US')
    search_items = search['tracks']['items']
    
    for i in range(len(search_items)):
        spotify_title = search_items[i]['name']
        spotify_artist = search_items[i]['artists'][0]['name']
        
        spotify_title_clean = re.sub(r"[,.;@#?!&$%()]+", ' ', spotify_title)
        spotify_title_clean = re.sub('\s+', ' ', title_clean).strip().lower()
        spotify_artist_clean = spotify_artist.lower().strip().lower()
        
        fuzzy_title_match = fuzz.token_set_ratio(title_clean, spotify_title_clean)
        fuzzy_artist_match = fuzz.token_set_ratio(artist_clean, spotify_artist_clean)
        fuzzy_match = (fuzzy_title_match + fuzzy_artist_match) / 2

        if (fuzzy_title_match >= 90) and (fuzzy_artist_match >= 50) and fuzzy_match >= 75:
            uri = search_items[i]['id']
            return uri
    return 0



In [ ]:
temp = list()

for i in tqdm_notebook(range(len(titles))):
    uri = get_spotify_uri(titles[i], artists[i])
    
    if uri != 0:
        temp.append(uri)
    else:
        temp.append(uri)
        errors.append(i)
    print(uri)
spotify_uri = spotify_uri + temp

In [ ]:
data['URI'] = spotify_uri
data = data[data.URI != 0]

In [85]:
data

,Artist,Title,Top100,URI
0,linkin park,in the end,0,60a0Rd6pjrkxjPbaKzXjfq
1,red hot chili peppers,by the way,0,1f2V8U1BiWaC9aJWmpOARe
2,evanescence,bring me to life,0,0COqiPhxzoWICwFCS4eZcp
3,thirty seconds to mars,the kill (bury me),0,54bzhkAeWNCzl6Reh4h8pV
4,muse,uprising,0,4VqPOruhp5EdPBeR92t6lQ
...,...,...,...,...
17494,bizarrap and shakira,"bzrp music sessions, vol. 53",1,4nrPB8O7Y7wsOCJdgXkthe
17495,travis scott,meltdown,1,67nepsnrcZkowTxMWigSbb
17496,latto,put it on da floor again,1,6c6WmIHcHlhccEwSFBhzNa
17497,lady gaga,bloody mary,1,11BKm0j4eYoCPPpCONAVwA


In [ ]:
data.to_csv('../Data/allSongs_w_uri.csv')

### Get Spotify Features

In [18]:
data = pd.read_csv('../Data/allSongs_w_uri.csv', index_col=0)

In [19]:
uris = list(data.URI)

danceability_list = list()
energy_list = list()
key_list = list()
loudness_list = list()
mode_list = list()
speechiness_list = list()
acousticness_list = list()
instrumentalness_list = list()
liveness_list = list()
valence_list = list()
tempo_list = list()
duration_list = list()
time_signature_list= list()

In [39]:
def get_audio_features(uri):
    try:
        search = sp.audio_features(uri)
        if search[0] == None:
            danceability_list.append(np.nan)
            energy_list.append(np.nan)
            key_list.append(np.nan)
            loudness_list.append(np.nan)
            mode_list.append(np.nan)
            speechiness_list.append(np.nan)
            acousticness_list.append(np.nan)
            instrumentalness_list.append(np.nan)
            liveness_list.append(np.nan)
            valence_list.append(np.nan)
            tempo_list.append(np.nan)
            duration_list.append(np.nan)
            time_signature_list.append(np.nan) 
            return ('Error on: ' + str(uri))
        print(search)
        search_list = search[0]
        
        danceability_list.append(search_list['danceability'])
        energy_list.append(search_list['energy'])
        key_list.append(search_list['key'])
        loudness_list.append(search_list['loudness'])
        mode_list.append(search_list['mode'])
        speechiness_list.append(search_list['speechiness'])
        acousticness_list.append(search_list['acousticness'])
        instrumentalness_list.append(search_list['instrumentalness'])
        liveness_list.append(search_list['liveness'])
        valence_list.append(search_list['valence'])
        tempo_list.append(search_list['tempo'])
        duration_list.append(search_list['duration_ms'])
        time_signature_list.append(search_list['time_signature'])
    except Exception as e:
        if e.args[0]==429:
            raise Exception
        else:
            pass

In [ ]:
for i in tqdm_notebook(range(len(uris)//100)):
    get_audio_features(uris[i:i+100])
get_audio_features(uris[17400:])

In [ ]:
data['Danceability'] = danceability_list
data['Energy'] = energy_list
data['Key'] = key_list
data['Loudness'] = loudness_list
data['Mode'] = mode_list
data['Speechiness'] = speechiness_list
data['Acousticness'] = acousticness_list
data['Instrumentalness'] = instrumentalness_list
data['Liveness'] = liveness_list
data['Valence'] = valence_list
data['Tempo'] = tempo_list
data['Duration'] = duration_list
data['Time_Signature'] = time_signature_list

In [ ]:
data = data.dropna()
data = data.drop_duplicates(subset="Title", keep="last")
data = data.reset_index(drop=True)
data.to_csv('songs_w_features.csv')

### Get Year


In [6]:
data = pd.read_csv("../Data/songs_w_features.csv", index_col=0)

In [7]:
songs = list(data.Title)
artists = list(data.Artist)
years = list()
errors = list()

In [8]:
def get_song_year(title, artist):
    title_clean = re.sub(r"[,.;@#?!&$%()]+", ' ', title)
    title_clean = re.sub('\s+', ' ', title_clean).strip()
    artist_clean = re.sub('\s+', ' ', artist).strip()
    
    query = title_clean + " " + artist_clean
    
    try:
        search = sp.search(q=query, limit=50, offset=0, type='track')
        search_items = search['tracks']['items']
        year = search_items[0]['album']['release_date']
        return year
    except Exception:
        year = 0
        return year

In [10]:
for i in tqdm_notebook(range(len(songs))):
    year = get_song_year(songs[i], artists[i])
    
    if year != 0 :
        years.append(year)
    else:
        years.append(year)
        print("Errored on " + str(i))
        errors.append(i)    

/tmp/ipykernel_122349/2074439663.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(songs))):


  0%|          | 0/17080 [00:00<?, ?it/s]

Errored on 186
Errored on 3707
Errored on 4866
Errored on 5838
Errored on 7805
Errored on 9093
Errored on 9131
Errored on 11877
Errored on 12121
Errored on 12263
Errored on 12264
Errored on 13118
Errored on 13686
Errored on 14648
Errored on 15494
Errored on 15578
Errored on 16280
Errored on 16750


In [11]:
new_years = list()

def clean_year(date):
    y = date.split('-')[0]
    return int(y)

In [12]:
for i in years:
    if i == 0:
        new_years.append(i)
    else:
        x = clean_year(i)
        new_years.append(x)

In [ ]:
data['Release_Year'] = new_years
data = data[data.Release_Year != 0]

In [19]:
data.to_csv('../Data/songs_w_features_year.csv')
data

,Artist,Title,Top100,URI,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration,Time_Signature,Release_Year
0,red hot chili peppers,by the way,0,1f2V8U1BiWaC9aJWmpOARe,0.451,0.970,0,-4.938,1,0.1070,0.026400,0.003550,0.1020,0.198,122.444,216933,4,2002
1,thirty seconds to mars,the kill (bury me),0,54bzhkAeWNCzl6Reh4h8pV,0.179,0.912,4,-3.881,0,0.0791,0.001400,0.000294,0.5820,0.289,182.990,231533,3,2019
2,muse,uprising,0,4VqPOruhp5EdPBeR92t6lQ,0.602,0.905,2,-4.046,1,0.0775,0.000202,0.064000,0.1170,0.411,128.019,304840,4,2009
3,limp bizkit,rollin' (air raid vehicle),0,3IV4swNduIRunHREK80owz,0.603,0.933,1,-3.358,1,0.1710,0.005910,0.000000,0.2060,0.709,96.306,213760,4,2000
4,rise against,savior,0,3asFGFY3uLjMDmML1p0tYm,0.339,0.931,5,-3.425,0,0.0620,0.001500,0.000142,0.4250,0.467,112.548,242280,4,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17075,bizarrap and shakira,"bzrp music sessions, vol. 53",1,4nrPB8O7Y7wsOCJdgXkthe,0.778,0.632,2,-5.600,0,0.0493,0.274000,0.000000,0.0915,0.498,122.104,218289,4,2023
17076,travis scott,meltdown,1,67nepsnrcZkowTxMWigSbb,0.557,0.774,7,-5.275,0,0.3510,0.012000,0.000000,0.3960,0.397,111.975,246134,4,2023
17077,latto,put it on da floor again,1,6c6WmIHcHlhccEwSFBhzNa,0.849,0.614,1,-7.323,1,0.3170,0.028200,0.000000,0.0603,0.472,111.363,185748,5,2023
17078,lady gaga,bloody mary,1,11BKm0j4eYoCPPpCONAVwA,0.591,0.637,9,-6.365,0,0.0300,0.010700,0.000002,0.1130,0.432,99.998,244760,4,2011
